In [ ]:
%load_ext autoreload
%autoreload 2

import os
# avoid thread conflicts between numpy and dask
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

from polsarpro.dev.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import open_netcdf_beam
from polsarpro.dev.io import read_psp_bin
from polsarpro.decompositions import tsvm
from polsarpro.dev.metrics import summarize_metrics, visualize_errors
import shutil

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/data_convert/"

# change to your data paths
# original dataset
input_alos_data = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")

# input files from C
input_test_dir = Path("/data/psp/SAN_FRANCISCO_ALOS1/")

# output files from C
output_test_dir = Path("/data/psp/res/tsvm_c/")

## Run the C-version on some test data

In [ ]:
# fl1: alpha, phi,tau, psi 
# fl2: alphas
# fl3: phis
# fl4: taus 
# fl5: psis

if not os.path.exists(output_test_dir):
    os.makedirs(output_test_dir)

input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: S2
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
fl1: 1
fl2: 1
fl3: 1
fl4: 1
fl5: 1
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"tsvm_decomposition.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_test_dir / "config.txt", output_test_dir)

## Load ALOS data and C outputs

In [ ]:
# uncomment to test on S matrix made with SNAP
S = open_netcdf_beam(input_alos_data)

## Apply the decomposition

In [ ]:
file_out = "/data/psp/res/test_tsvm.nc"
# netcdf writer cannot overwrite
if os.path.isfile(file_out):
    os.remove(file_out)

flags = ("alpha_phi_tau_psi", "alpha", "phi", "tau", "psi")

with ProgressBar():
    # uncomment to compute and store result in a file
    tsvm(S, boxcar_size=[7, 7], flags=flags).to_netcdf(file_out)

# Numerical evaluation

In [ ]:
# open python output (comment if using compute)
out_py = xr.open_dataset("/data/psp/res/test_tsvm.nc")
# open C-PolSARPro outputs
out_c = {}
out_names = []
if "alpha_phi_tau_psi" in flags:
    out_names.extend(["alpha_s", "phi_s", "tau_m", "psi"])
if "alpha" in flags:
    out_names.extend(["alpha_s1", "alpha_s2", "alpha_s3"])
if "phi" in flags:
    out_names.extend(["phi_s1", "phi_s2", "phi_s3"])
if "tau" in flags:
    out_names.extend(["tau_m1", "tau_m2", "tau_m3"])
if "psi" in flags:
    out_names.extend(["psi1", "psi2", "psi3"])

for name in out_names:
    file_name = output_test_dir / f"TSVM_{name}.bin"
    out_c[name] = read_psp_bin(file_name)

df = summarize_metrics(out_py, out_c, short_titles=False)
df

In [ ]:
visualize_errors(out_py=out_py, out_c=out_c, clip=False)